In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf

import geoviews.tile_sources as gts
import cartopy
from cartopy import crs as ccrs

from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import WMTSTileSource

import pyproj

hv.extension('bokeh')

In [ ]:
%%output size=150

#Data gathering
headers = ['date','canceled']
dates = ['date']
cfpd = pd.read_csv('results/cfpd.csv', names=headers, parse_dates=dates)

cfpd['year'] = cfpd['date'].dt.year
cfpd['month'] = cfpd['date'].dt.month

#Plot

#Method used to return a different plot for each year. It's then used by the HoloMap (the slider)
def year_bar(year):
    cfpdy = cfpd[cfpd['year'] == year]
    return hv.BoxWhisker(cfpdy, kdims=['month'],vdims='canceled').options(logy=True)

years = {y : year_bar(y) for y in cfpd.year.unique() }

hv.HoloMap(years, kdims = ['year'])

In [ ]:
%%output size=150

#Data gathering
headers = ['date','canceled']
dates = ['date']
pwcpw = pd.read_csv('results/pwcpw.csv', names=headers, parse_dates=dates)

pwcpw['year'] = pwcpw['date'].dt.year
pwcpw['month'] = pwcpw['date'].dt.month

#We don't have data before 2003
pwcpw = pwcpw[pwcpw['year'] >= 2003]

#Plot
hv.Curve(pwcpw, vdims=['canceled']).options(width=600)

In [ ]:
%%output size=150
%%opts Curve  [height=100 width=600]
headers = ['group','score']
pddhpg = pd.read_csv('results/pddhpg.csv', names=headers)
pddhpg_viz = hv.Bars(pddhpg).options(width=600, color='green')
pddhpg_viz

In [ ]:
#Data gathering (airports dataset for lat/long, ppa_res for the score data)

airports = pd.read_csv('airports.csv', header=0)

headers = ['date', 'iata', 'score']
dates = ['date']
ppa_res = pd.read_csv('results/ppa.csv', names=headers, parse_dates=dates)
ppa_res = ppa_res.groupby([ppa_res['date'].dt.year,'iata']).mean().reset_index()

ppa_res = ppa_res.rename(columns={'date':'Year'})

airports['IATA'] = airports['IATA'].map(lambda x: x.strip())
ppa_res['iata'] = ppa_res['iata'].map(lambda x: x.strip())

#Join airport/score to get lat, long, year and score for each airport

ppa = ppa_res.set_index('iata').join(airports.set_index('IATA'))
gvppa = gv.Dataset(ppa, kdims=['score','Name','City','Country','Year'])

#Plot: convert the dataset to geoviews and then plot it
# The 'year' field is left out of the dimensions, so a slider/HoloMap is inferred

tile = gts.OSM

points = gvppa.to(gv.Points,['Longitude', 'Latitude'],['score','Name','City','Country'], crs=ccrs.PlateCarree())

ppagraph = tile * points.options(width=600, height=600, tools=['hover'], size_index=2, color_index=2, size=1, cmap='viridis')

ppagraph

In [ ]:
%%opts Chord [edge_color_index='dep' label_index='City' color_index='iata' width=600 height=600]
%%opts Chord (cmap='Category20' edge_cmap='Category20')
#Data gathering

airports = pd.read_csv('airports.csv', header=0)
airports = airports[['IATA','City']]

headers = ['dep', 'arr', 'year', 'month','count']
fppm_res = pd.read_csv('results/fppm.csv', names=headers)

airports['IATA'] = airports['IATA'].map(lambda x: x.strip())
fppm_res['dep'] = fppm_res['dep'].map(lambda x: x.strip())
fppm_res['arr'] = fppm_res['arr'].map(lambda x: x.strip())

depcount = fppm_res.drop_duplicates(subset=['dep'])['dep'].count()
arrcount = fppm_res.drop_duplicates(subset=['arr'])['arr'].count()
drop = 'dep' if depcount > arrcount else 'arr'

nodes = fppm_res.drop_duplicates(subset=[drop]).set_index(drop).join(airports.set_index('IATA'))
nodes = nodes.reset_index().rename(columns = {'index' :'iata'})[['iata','City']]
nodes = hv.Dataset(nodes,'iata')

#Plotting data

#HoloMap helper function

def map_yearmonth(year, month, howmany):
    chordsym = fppm_res[(fppm_res['year']==year) & (fppm_res['month']==month)]
    
    chord = hv.Chord((chordsym,nodes),['dep','arr'],['count'])
    busiest = list(chordsym.groupby('dep').count().sort_values('count').iloc[-howmany:].index.values)
    busiest_airports = chord.select(iata=busiest, selection_mode='nodes')
    
    return busiest_airports

busiest = 20

#Should be done for all the years with a HoloMap, but it crashes (too much data)

#graphs = {(y,m) : map_yearmonth(y,m,busiest) for y in fppm_res.year.unique() for m in fppm_res.month.unique()}

#hv.HoloMap(graphs, kdims = ['year','month'])

map_yearmonth(2003,2, 10)